In [1]:
# import libraries
import pandas as pd
import requests
import numpy as np
import re

In [2]:
# To view all the 81 columns in the data frame, use the set_option to display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## USDA Food and Nutrition data files downloaded from the link - 
### https://www.ars.usda.gov/ARSUserFiles/80400525/Data/SR-Legacy/SR-Leg_ASC.zip

In [3]:
# Get the list of data files to be loaded 
files = '''DATA_SRC.txt:     text/plain; charset=us-ascii
DATSRCLN.txt:     text/plain; charset=us-ascii
DERIV_CD.txt:     text/plain; charset=us-ascii
FD_GROUP.txt:     text/plain; charset=us-ascii
FOOD_DES.txt:     text/plain; charset=iso-8859-1
FOOTNOTE.txt:     text/plain; charset=iso-8859-1
FOOTNOTE_out.txt: text/plain; charset=us-ascii
LANGDESC.txt:     text/plain; charset=us-ascii
LANGUAL.txt:      text/plain; charset=us-ascii
NUTR_DEF.txt:     text/plain; charset=iso-8859-1
NUT_DATA.txt:     text/plain; charset=us-ascii
SRC_CD.txt:       text/plain; charset=us-ascii
WEIGHT.txt:       text/plain; charset=us-ascii'''.split('\n')

In [4]:
files


['DATA_SRC.txt:     text/plain; charset=us-ascii',
 'DATSRCLN.txt:     text/plain; charset=us-ascii',
 'DERIV_CD.txt:     text/plain; charset=us-ascii',
 'FD_GROUP.txt:     text/plain; charset=us-ascii',
 'FOOD_DES.txt:     text/plain; charset=iso-8859-1',
 'FOOTNOTE.txt:     text/plain; charset=iso-8859-1',
 'FOOTNOTE_out.txt: text/plain; charset=us-ascii',
 'LANGDESC.txt:     text/plain; charset=us-ascii',
 'LANGUAL.txt:      text/plain; charset=us-ascii',
 'NUTR_DEF.txt:     text/plain; charset=iso-8859-1',
 'NUT_DATA.txt:     text/plain; charset=us-ascii',
 'SRC_CD.txt:       text/plain; charset=us-ascii',
 'WEIGHT.txt:       text/plain; charset=us-ascii']

In [5]:
files[0]

'DATA_SRC.txt:     text/plain; charset=us-ascii'

In [6]:
test = files[0]
regex = r"^(.+txt).*charset=(.+)$"
re.findall(regex, test)

[('DATA_SRC.txt', 'us-ascii')]

In [7]:
# Using regex
encodings = [re.findall(regex, file_)[0] for file_ in files ]
encodings

[('DATA_SRC.txt', 'us-ascii'),
 ('DATSRCLN.txt', 'us-ascii'),
 ('DERIV_CD.txt', 'us-ascii'),
 ('FD_GROUP.txt', 'us-ascii'),
 ('FOOD_DES.txt', 'iso-8859-1'),
 ('FOOTNOTE.txt', 'iso-8859-1'),
 ('FOOTNOTE_out.txt', 'us-ascii'),
 ('LANGDESC.txt', 'us-ascii'),
 ('LANGUAL.txt', 'us-ascii'),
 ('NUTR_DEF.txt', 'iso-8859-1'),
 ('NUT_DATA.txt', 'us-ascii'),
 ('SRC_CD.txt', 'us-ascii'),
 ('WEIGHT.txt', 'us-ascii')]

In [8]:
encodings_dict = {key: val for key,val in encodings }
encodings_dict

{'DATA_SRC.txt': 'us-ascii',
 'DATSRCLN.txt': 'us-ascii',
 'DERIV_CD.txt': 'us-ascii',
 'FD_GROUP.txt': 'us-ascii',
 'FOOD_DES.txt': 'iso-8859-1',
 'FOOTNOTE.txt': 'iso-8859-1',
 'FOOTNOTE_out.txt': 'us-ascii',
 'LANGDESC.txt': 'us-ascii',
 'LANGUAL.txt': 'us-ascii',
 'NUTR_DEF.txt': 'iso-8859-1',
 'NUT_DATA.txt': 'us-ascii',
 'SRC_CD.txt': 'us-ascii',
 'WEIGHT.txt': 'us-ascii'}

In [9]:
filename = "WEIGHT.txt"
encoding=encodings_dict[filename]

In [10]:
# Get the File Directory
filename = "WEIGHT.txt"
dir = "SR-Leg_ASC"
file_path = "./{}/{}".format(dir, filename)
print(file_path)

./SR-Leg_ASC/WEIGHT.txt


In [11]:
filename = "WEIGHT.txt"
txt=filename[:-4]
print(txt)

WEIGHT


In [12]:
for i in encodings_dict:
    print (i, ' - ', encodings_dict[i])

DATA_SRC.txt  -  us-ascii
DATSRCLN.txt  -  us-ascii
DERIV_CD.txt  -  us-ascii
FD_GROUP.txt  -  us-ascii
FOOD_DES.txt  -  iso-8859-1
FOOTNOTE.txt  -  iso-8859-1
FOOTNOTE_out.txt  -  us-ascii
LANGDESC.txt  -  us-ascii
LANGUAL.txt  -  us-ascii
NUTR_DEF.txt  -  iso-8859-1
NUT_DATA.txt  -  us-ascii
SRC_CD.txt  -  us-ascii
WEIGHT.txt  -  us-ascii


## Read and create dataframes for each file; remove unwanted characters
The ASCII files are delimited as follows: All fields are separated by carets (^), and text fields
are surrounded by tildes (~). A double caret (^^) or two carets and two tildes (~~)
appear when a value is null or a field is blank

In [13]:
dfs = {}
dir = "SR-Leg_ASC"
for i in encodings_dict:
    filename=i
    encoding = encodings_dict[filename]
    file_path = "./{}/{}".format(dir, filename)
    txt=filename[:-4]
    dfs[txt] = pd.read_csv(file_path, encoding = encoding, sep="^", header=None)
    dfs[txt].replace(regex=True,inplace=True,to_replace=r'~',value=r'')

/Users/kkesari/anaconda3/envs/dsi/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Datafile Name  - format -    Description

* DATA_SRC.txt  -  us-ascii    - Sources of Data (DATA_SRC) File

* DATSRCLN.txt  -  us-ascii    - Sources of Data Link (DATSRCLN) File 

* DERIV_CD.txt  -  us-ascii    - Data Derivation Code Description (DERIV_CD) File

* FD_GROUP.txt  -  us-ascii    - Food Group Description

* FOOD_DES.txt  -  iso-8859-1  - Food Description (FOOD_DES) File

* FOOTNOTE.txt  -  iso-8859-1  - Footnote (FOOTNOTE) File

* FOOTNOTE_out.txt  -  us-ascii - Footnote (FOOTNOTE) File

* LANGDESC.txt  -  us-ascii   - LanguaL Factor Description (LANGDESC) File

* LANGUAL.txt  -  us-ascii    - LanguaL Factor (LANGUAL) File

* NUTR_DEF.txt  -  iso-8859-1 - Nutrient Definition (NUTR_DEF) File

* NUT_DATA.txt  -  us-ascii   - Nutrient Data (NUT_DATA) File

* SRC_CD.txt  -  us-ascii     - Source Code (SRC_CD) File

* WEIGHT.txt  -  us-ascii     - Weight (WEIGHT) File


In [14]:
#  Set the column names for each of the dataframes

dfs['DATA_SRC'].columns = ['DataSrc_ID', 'Authors','Title','Year','Journal',
                           'Vol_City','Issue_State','Start_Page','End_Page']

dfs['DATSRCLN'].columns = ['NDB_No','Nutr_No','DataSrc_ID']
dfs['DERIV_CD'].columns = ['Deriv_Cd','Deriv_Desc']
dfs['FD_GROUP'].columns = ['FdGrp_Cd','FdGrp_Desc']
dfs['FOOD_DES'].columns = ['NDB_No','FdGrp_Cd','Long_Desc','Shrt_Desc','ComName','ManufacName','Survey',
                           'Ref_desc','Refuse','SciName','N_Factor','Pro_Factor','Fat_Factor','CHO_Factor']
dfs['FOOTNOTE'].columns = ['NDB_No','Footnt_No','Footnt_Typ','Nutr_No','Footnt_Txt']
dfs['FOOTNOTE_out'].columns = ['NDB_No','Footnt_No','Footnt_Typ','Nutr_No','Footnt_Txt']
dfs['LANGDESC'].columns = ['Factor_Code','Description']
dfs['LANGUAL'].columns = ['NDB_No','Factor_Code']
dfs['NUTR_DEF'].columns = ['Nutr_No','Units','Tagname','NutrDesc','Num_Dec','SR_Order']
dfs['NUT_DATA'].columns = ['NDB_No','Nutr_No','Nutr_Val','Num_Data_Pts','Std_Error','Src_Cd','Deriv_Cd',
                           'Ref_NDB_No','Add_Nutr_Mark','Num_Studies','Min','Max','DF','Low_EB','Up_EB',
                           'Stat_cmt','AddMod_Date']
dfs['SRC_CD'].columns = ['Src_Cd','SrcCd_Desc']
dfs['WEIGHT'].columns = ['NDB_No','Seq','Amount','Msre_Desc','Gm_Wgt','Num_Data_Pts','Std_Dev']

In [15]:
print('**** DATA_SRC ****')
print(dfs['DATA_SRC'].dtypes, dfs['DATA_SRC'].shape)
print('*' * 20)

print('**** DATSRCLN ****')
print(dfs['DATSRCLN'].dtypes, dfs['DATSRCLN'].shape)
print('*' * 20)

print('**** DERIV_CD ****')
print(dfs['DERIV_CD'].dtypes,dfs['DERIV_CD'].shape)
print('*' * 20)

print('**** FD_GROUP ****')
print(dfs['FD_GROUP'].dtypes,dfs['FD_GROUP'].shape)
print('*' * 20)

print('**** FOOD_DES ****')
print(dfs['FOOD_DES'].dtypes,dfs['FOOD_DES'].shape)
print('*' * 20)

print('**** FOOTNOTE ****')
print(dfs['FOOTNOTE'].dtypes,dfs['FOOTNOTE'].shape)
print('*' * 20)

print('**** FOOTNOTE_out ****')
print(dfs['FOOTNOTE_out'].dtypes, dfs['FOOTNOTE_out'].shape)
print('*' * 20)

print('**** LANGDESC ****')
print(dfs['LANGDESC'].dtypes, dfs['LANGDESC'].shape)
print('*' * 20)

print('**** LANGUAL ****')
print(dfs['LANGUAL'].dtypes, dfs['LANGUAL'].shape)
print('*' * 20)

print('**** NUTR_DEF ****')
print(dfs['NUTR_DEF'].dtypes, dfs['NUTR_DEF'].shape)
print('*' * 20)

print('**** NUT_DATA ****')
print(dfs['NUT_DATA'].dtypes, dfs['NUT_DATA'].shape)
print('*' * 20)

print('**** SRC_CD ****')
print(dfs['SRC_CD'].dtypes, dfs['SRC_CD'].shape)
print('*' * 20)

print('**** WEIGHT ****')
print(dfs['WEIGHT'].dtypes, dfs['WEIGHT'].shape)
print('*' * 20)

**** DATA_SRC ****
DataSrc_ID     object
Authors        object
Title          object
Year           object
Journal        object
Vol_City       object
Issue_State    object
Start_Page     object
End_Page       object
dtype: object (606, 9)
********************
**** DATSRCLN ****
NDB_No        object
Nutr_No       object
DataSrc_ID    object
dtype: object (228457, 3)
********************
**** DERIV_CD ****
Deriv_Cd      object
Deriv_Desc    object
dtype: object (57, 2)
********************
**** FD_GROUP ****
FdGrp_Cd      object
FdGrp_Desc    object
dtype: object (25, 2)
********************
**** FOOD_DES ****
NDB_No          object
FdGrp_Cd        object
Long_Desc       object
Shrt_Desc       object
ComName         object
ManufacName     object
Survey          object
Ref_desc        object
Refuse         float64
SciName         object
N_Factor       float64
Pro_Factor     float64
Fat_Factor     float64
CHO_Factor     float64
dtype: object (7793, 14)
********************
**** FOOTNOTE *

### Review each of the data frames using head(3)

In [16]:
dfs['DATA_SRC'].head(3)

,DataSrc_ID,Authors,Title,Year,Journal,Vol_City,Issue_State,Start_Page,End_Page
0,D2028,"A. Sanchez-Marroquin, A. Feria-Morales, S. May...","Processing, nutritional quality and sensory ev...",1987,Journal of Food Science,52,5,NaN,NaN
1,D3296,"C.E. Damon, B.C. Pettitt",Sugars and Sugar Products: High Performance Li...,1980,J. AOAC,63,3,476,480
2,D3315,"W.J. Hurst, R.A. Martin, B.L. Zoumas",Application of HPLC to Characterization of Ind...,1979,Journal of Food Science,44,NaN,892,893


In [17]:
dfs['DATSRCLN'].head(3)

,NDB_No,Nutr_No,DataSrc_ID
0,01001,301,S10
1,01001,301,S11
2,01001,301,S12


In [18]:
dfs['DERIV_CD'].head(3)

,Deriv_Cd,Deriv_Desc
0,A,Analytical data
1,AI,Analytical data; from the literature or gover...
2,AR,Analytical data; derived by linear regression


In [19]:
dfs['FD_GROUP'].head(3)

,FdGrp_Cd,FdGrp_Desc
0,0100,Dairy and Egg Products
1,0200,Spices and Herbs
2,0300,Baby Foods


In [20]:
dfs['FOOD_DES'].head(3)

,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,ComName,ManufacName,Survey,Ref_desc,Refuse,SciName,N_Factor,Pro_Factor,Fat_Factor,CHO_Factor
0,01001,0100,"Butter, salted","BUTTER,WITH SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87
1,01002,0100,"Butter, whipped, with salt","BUTTER,WHIPPED,W/ SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,NaN,NaN,NaN
2,01003,0100,"Butter oil, anhydrous","BUTTER OIL,ANHYDROUS",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87


In [21]:
dfs['FOOTNOTE'].head(3)

,NDB_No,Footnt_No,Footnt_Typ,Nutr_No,Footnt_Txt
0,09129,01,D,NaN,"Includes analysis of Higgins, Jumbo, and Roano..."
1,09130,01,D,NaN,Includes purple and white grape juice
2,09135,01,D,NaN,Includes purple and white grape juice


In [22]:
dfs['FOOTNOTE_out'].head(3)

,NDB_No,Footnt_No,Footnt_Typ,Nutr_No,Footnt_Txt
0,09129,01,D,NaN,"Includes analysis of Higgins, Jumbo, and Roano..."
1,09130,01,D,NaN,Includes purple and white grape juice
2,09135,01,D,NaN,Includes purple and white grape juice


In [23]:
dfs['LANGDESC'].head(3)

,Factor_Code,Description
0,A0107,"BAKERY PRODUCT, UNSWEETENED (US CFR)"
1,A0113,SPICE OR HERB (US CFR)
2,A0125,GRAIN OR STARCH PRODUCT (US CFR)


In [24]:
dfs['LANGUAL'].head(3)

,NDB_No,Factor_Code
0,02001,A0113
1,02001,A1272
2,02001,B1356


In [25]:
dfs['NUTR_DEF'].head(3)

,Nutr_No,Units,Tagname,NutrDesc,Num_Dec,SR_Order
0,203,g,PROCNT,Protein,2,600
1,204,g,FAT,Total lipid (fat),2,800
2,205,g,CHOCDF,"Carbohydrate, by difference",2,1100


In [26]:
dfs['NUT_DATA'].head(3)

,NDB_No,Nutr_No,Nutr_Val,Num_Data_Pts,Std_Error,Src_Cd,Deriv_Cd,Ref_NDB_No,Add_Nutr_Mark,Num_Studies,Min,Max,DF,Low_EB,Up_EB,Stat_cmt,AddMod_Date
0,01001,208,717.0,0,NaN,4,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/01/2010
1,01001,262,0.0,0,NaN,7,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/01/2001
2,01001,263,0.0,0,NaN,7,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/01/2001


In [27]:
dfs['SRC_CD'].head(3)

,Src_Cd,SrcCd_Desc
0,1,Analytical or derived from analytical
1,11,Aggregated data involving comb. of codes other...
2,12,Manufacturer's analytical; partial documentation


In [28]:
dfs['WEIGHT'].head(3)

,NDB_No,Seq,Amount,Msre_Desc,Gm_Wgt,Num_Data_Pts,Std_Dev
0,01001,1,1.0,"pat (1"" sq, 1/3"" high)",5.0,NaN,NaN
1,01001,2,1.0,tbsp,14.2,NaN,NaN
2,01001,3,1.0,cup,227.0,NaN,NaN


###  Convert  identified columns as INT  as per the document listed.

In [29]:
# Convert some of the identified columns as INT and FLOAT as per the document listed.

# Convert the identified columns into int and float of the dataframe 'NUT_DATA'.
dfs['NUT_DATA']['Src_Cd'] = dfs['NUT_DATA']['Src_Cd'].astype(int)

# Convert the identified columns into int of the dataframe 'SRC_CD'.
dfs['SRC_CD']['Src_Cd'] = dfs['SRC_CD']['Src_Cd'].astype(int)





In [30]:
# Merge the two dataframes - FOOD_DES and FD_GROUP on FdGrp_Cd
df_food_fdg = pd.merge(how ='left',
         left = dfs['FOOD_DES'],
         right = dfs['FD_GROUP'],
         on = 'FdGrp_Cd')

In [31]:
df_food_fdg.shape

(7793, 15)

In [32]:
df_food_fdg.head(3)

,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,ComName,ManufacName,Survey,Ref_desc,Refuse,SciName,N_Factor,Pro_Factor,Fat_Factor,CHO_Factor,FdGrp_Desc
0,01001,0100,"Butter, salted","BUTTER,WITH SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87,Dairy and Egg Products
1,01002,0100,"Butter, whipped, with salt","BUTTER,WHIPPED,W/ SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,NaN,NaN,NaN,Dairy and Egg Products
2,01003,0100,"Butter oil, anhydrous","BUTTER OIL,ANHYDROUS",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87,Dairy and Egg Products


In [33]:
df_food_fdg[['FdGrp_Cd','FdGrp_Desc']].groupby(by='FdGrp_Cd').count().sort_index( ascending=False).head()



,FdGrp_Desc
FdGrp_Cd,
3600,109
3500,165
2500,176
2200,81
2100,312


In [34]:
# Merge the two dataframes -nut_data and nutrient_def on nutrient number

df_nut_data_def = pd.merge(how ='left',
                          left = dfs['NUT_DATA'],
                          right = dfs['NUTR_DEF'],
                          on = 'Nutr_No')


In [35]:
df_nut_data_def.shape

(644125, 22)

In [36]:
df_nut_data_def.head(3)

,NDB_No,Nutr_No,Nutr_Val,Num_Data_Pts,Std_Error,Src_Cd,Deriv_Cd,Ref_NDB_No,Add_Nutr_Mark,Num_Studies,Min,Max,DF,Low_EB,Up_EB,Stat_cmt,AddMod_Date,Units,Tagname,NutrDesc,Num_Dec,SR_Order
0,01001,208,717.0,0,NaN,4,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/01/2010,kcal,ENERC_KCAL,Energy,0,300
1,01001,262,0.0,0,NaN,7,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/01/2001,mg,CAFFN,Caffeine,0,18300
2,01001,263,0.0,0,NaN,7,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/01/2001,mg,THEBRN,Theobromine,0,18400


In [37]:
df_nut_data_def[['Nutr_No','NutrDesc']].groupby(by='Nutr_No').count().sort_index( ascending=False).head()

,NutrDesc
Nutr_No,
859,168
858,1069
857,114
856,568
855,165


In [38]:
# Merge the two dataframes - DATSRCLN  and DATA_SRC on DataSrc ID

df_SRC_SRCLN = pd.merge(how ='left',
                        left = dfs['DATA_SRC'],
                        right = dfs['DATSRCLN'],
                        on = 'DataSrc_ID')


In [39]:
df_SRC_SRCLN.shape

(228457, 11)

In [40]:
df_SRC_SRCLN.head(3)

,DataSrc_ID,Authors,Title,Year,Journal,Vol_City,Issue_State,Start_Page,End_Page,NDB_No,Nutr_No
0,D2028,"A. Sanchez-Marroquin, A. Feria-Morales, S. May...","Processing, nutritional quality and sensory ev...",1987,Journal of Food Science,52,5,NaN,NaN,20017,203
1,D2028,"A. Sanchez-Marroquin, A. Feria-Morales, S. May...","Processing, nutritional quality and sensory ev...",1987,Journal of Food Science,52,5,NaN,NaN,20017,204
2,D2028,"A. Sanchez-Marroquin, A. Feria-Morales, S. May...","Processing, nutritional quality and sensory ev...",1987,Journal of Food Science,52,5,NaN,NaN,20017,207


In [41]:
# Merge the two dataframes - LANGUAL  and LANGDESC on Factor_Code
df_LANGUAL_DESC = pd.merge(how ='left',
                           left = dfs['LANGUAL'],
                           right = dfs['LANGDESC'],
                           on = 'Factor_Code')


In [42]:
df_LANGUAL_DESC.shape

(37910, 3)

In [43]:
df_LANGUAL_DESC.head(3)

,NDB_No,Factor_Code,Description
0,02001,A0113,SPICE OR HERB (US CFR)
1,02001,A1272,0200 SPICES AND HERBS (USDA SR)
2,02001,B1356,ALLSPICE


In [44]:
df_LANGUAL_DESC['NDB_No'].value_counts().head()

05092    25
05101    25
05067    24
05058    24
05030    24
Name: NDB_No, dtype: int64

In [45]:
# Sample a record with NDB_No '02001'
df_food_fdg[df_food_fdg['NDB_No']=='02001']

,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,ComName,ManufacName,Survey,Ref_desc,Refuse,SciName,N_Factor,Pro_Factor,Fat_Factor,CHO_Factor,FdGrp_Desc
263,02001,0200,"Spices, allspice, ground","ALLSPICE,GROUND",NaN,NaN,NaN,NaN,0.0,Pimenta dioica,6.25,3.36,8.37,2.35,Spices and Herbs


In [46]:
df_LANGUAL_DESC[df_LANGUAL_DESC['NDB_No']=='02001'].head(3)

,NDB_No,Factor_Code,Description
0,02001,A0113,SPICE OR HERB (US CFR)
1,02001,A1272,0200 SPICES AND HERBS (USDA SR)
2,02001,B1356,ALLSPICE


In [47]:
df_food_fdg.shape

(7793, 15)

In [48]:
df_food_fdg.columns

Index(['NDB_No', 'FdGrp_Cd', 'Long_Desc', 'Shrt_Desc', 'ComName', 'ManufacName', 'Survey', 'Ref_desc', 'Refuse', 'SciName', 'N_Factor', 'Pro_Factor', 'Fat_Factor', 'CHO_Factor', 'FdGrp_Desc'], dtype='object')

In [49]:
#  View all the food_des data with food group code as '0100'
dfs['FOOD_DES'][dfs['FOOD_DES']['FdGrp_Cd']=='0100'].head(3)

,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,ComName,ManufacName,Survey,Ref_desc,Refuse,SciName,N_Factor,Pro_Factor,Fat_Factor,CHO_Factor
0,01001,0100,"Butter, salted","BUTTER,WITH SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87
1,01002,0100,"Butter, whipped, with salt","BUTTER,WHIPPED,W/ SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,NaN,NaN,NaN
2,01003,0100,"Butter oil, anhydrous","BUTTER OIL,ANHYDROUS",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87


In [50]:
# different food groups 
df_food_fdg['FdGrp_Desc'].value_counts()

Beef Products                          954
Vegetables and Vegetable Products      814
Baked Products                         517
Lamb, Veal, and Game Products          464
Poultry Products                       383
Beverages                              366
Sweets                                 358
Fruits and Fruit Juices                355
Baby Foods                             345
Pork Products                          336
Fast Foods                             312
Dairy and Egg Products                 291
Legumes and Legume Products            290
Finfish and Shellfish Products         264
Soups, Sauces, and Gravies             254
Fats and Oils                          216
Breakfast Cereals                      195
Cereal Grains and Pasta                181
Snacks                                 176
Sausages and Luncheon Meats            167
American Indian/Alaska Native Foods    165
Nut and Seed Products                  137
Restaurant Foods                       109
Meals, Entr

In [51]:
# all foods that belong to restaraunt foods
df_food_fdg[df_food_fdg['FdGrp_Desc']=='Restaurant Foods'].head(3)

,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,ComName,ManufacName,Survey,Ref_desc,Refuse,SciName,N_Factor,Pro_Factor,Fat_Factor,CHO_Factor,FdGrp_Desc
7462,36000,3600,"APPLEBEE'S, 9 oz house sirloin steak","APPLEBEE'S,9 OZ HOUSE SIRLOIN STEAK","family style, applebees",Applebee's,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Restaurant Foods
7463,36001,3600,"APPLEBEE'S, Double Crunch Shrimp","APPLEBEE'S,DOUBLE CRUNCH SHRIMP","family style, applebees",Applebee's,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Restaurant Foods
7464,36002,3600,"APPLEBEE'S, french fries","APPLEBEE'S,FRENCH FR","family style, applebees",Applebee's,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Restaurant Foods


In [52]:
df_food_fdg[(df_food_fdg['FdGrp_Desc']=='Restaurant Foods')]['ManufacName'].value_counts()

Denny's                     12
Applebee's                  11
Cracker Barrel              10
T.G.I Friday's               7
Carrabba's Italian Grill     5
Darden Group                 5
Name: ManufacName, dtype: int64

In [53]:
# all foods that belong to restaraunt foods - APPLEBEE'S,FRENCH FR
df_food_fdg[((df_food_fdg['FdGrp_Desc']=='Restaurant Foods') & (df_food_fdg['NDB_No']=='36002'))].head()


,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,ComName,ManufacName,Survey,Ref_desc,Refuse,SciName,N_Factor,Pro_Factor,Fat_Factor,CHO_Factor,FdGrp_Desc
7464,36002,3600,"APPLEBEE'S, french fries","APPLEBEE'S,FRENCH FR","family style, applebees",Applebee's,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Restaurant Foods


In [54]:
# find the nutrition values in APPLEBEE'S,FRENCH FR

df_nut_data_def[df_nut_data_def['NDB_No']=='36002'].head(3)

,NDB_No,Nutr_No,Nutr_Val,Num_Data_Pts,Std_Error,Src_Cd,Deriv_Cd,Ref_NDB_No,Add_Nutr_Mark,Num_Studies,Min,Max,DF,Low_EB,Up_EB,Stat_cmt,AddMod_Date,Units,Tagname,NutrDesc,Num_Dec,SR_Order
144752,36002,208,290.0,0,NaN,4,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/01/2013,kcal,ENERC_KCAL,Energy,0,300
144753,36002,268,1213.0,0,NaN,4,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/01/2013,kJ,ENERC_KJ,Energy,0,400
144754,36002,301,19.0,6,1.0,1,A,NaN,NaN,1.0,16.0,23.0,5.0,16.0,21.0,"2, 3",03/01/2013,mg,CA,"Calcium, Ca",0,5300


In [55]:
dfs['WEIGHT'][dfs['WEIGHT']['NDB_No']=='36002']

,NDB_No,Seq,Amount,Msre_Desc,Gm_Wgt,Num_Data_Pts,Std_Dev
13866,36002,1,1.0,serving,164.0,12.0,32.16


In [56]:
fast_foods_df=df_food_fdg[df_food_fdg['FdGrp_Desc']=='Fast Foods']

In [57]:
# Get Cholesterol for Fast foods, biscuit, with egg and bacon
df_nut_data_def[(df_nut_data_def['NDB_No']=='21003') &(df_nut_data_def['NutrDesc']=='Cholesterol')]['Nutr_Val']

119640    235.0
Name: Nutr_Val, dtype: float64

In [58]:
# load food and nutrient description data frames to csv
df_food_fdg.to_csv('./prepared_data/food_descr.csv')
df_nut_data_def.to_csv('./prepared_data/nutr_descr.csv')

In [59]:
# fast_foods_df.to_csv('./prepared_data/fast_foods_test.csv')

In [60]:
# example where food group is 'Restaurant Foods' and food item is 'fries'.
                                                                         
french_fries_df=df_food_fdg[(df_food_fdg['FdGrp_Desc']=='Restaurant Foods')& (df_food_fdg['Long_Desc'].str.find('fries')>0)][['NDB_No',
                                                                                                                             'Long_Desc',
                                                                                                                             'Shrt_Desc',
                                                                                                                             'ComName',
                                                                                                                             'ManufacName',
                                                                                                                            ]]  
french_fries_df
                                                                         
                                                                         
                                                                         

,NDB_No,Long_Desc,Shrt_Desc,ComName,ManufacName
7464,36002,"APPLEBEE'S, french fries","APPLEBEE'S,FRENCH FR","family style, applebees",Applebee's
7469,36007,"T.G.I. FRIDAY'S, french fries","T.G.I. FRIDAY'S,FRENCH FR","family style, Fridays, TGI Friday's",T.G.I Friday's
7476,36014,"Restaurant, family style, french fries","RESTAURANT,FAMILY STYLE,FRENCH FR",family style,NaN
7547,36607,"CRACKER BARREL, steak fries","CRACKER BARREL,STEAK FRIES","french, family style",Cracker Barrel
7550,36610,"DENNY'S, french fries","DENNY'S,FRENCH FR",family style,Denny's
